In [12]:
import pandas as pd

import os

import warnings
warnings.filterwarnings('ignore')

**Februari 2019**

Örjan fick idén att kolla upp i vilka kommuner där kommunen och landstingen är viktigaste arbetsgivare. Siffrorna kommer från SCB (RAMS, dvs registerdata) och vi valde att kolla i förhållande till nattbefolkningen, dvs de som är förvärvsarbetande och som bor i kommunen. [Tabellerna hittar man här!](http://www.statistikdatabasen.scb.se/pxweb/sv/ssd/START__AM__AM0207__AM0207J/NattSektAldK/?rxid=daebc44d-0582-4bee-8c56-ce4bde1f7e03)

Det här måttet bör ses som ett någorlunda "trubbigt". Eftersom vi undersöker nattbefolkningen så kontrollerar vi hur många av de som bor i kommunen som jobbar i offentlig verksamhet (minus statliga myndigheter då). Men dessa boende kan i praktiken arbeta i en grannkommuns verksamhet under dagtid. Kommuner i storstadsområdena blir därför ganska ointressanta. Folk ifrån hela Storstockholmsområdet kan t ex arbeta på Nya Karolinska sjukhuset i Solna.

Vi valde därför att endast fokusera på de som har ***högst andel*** som jobbar inom kommunal och regional verksamhet. Ifall man har en relativt hög andel boende i kommunen som arbetar för en kommun eller ett landsting så ringar det in det vi vill berätta om: hur viktig kommunala och regionala statsapparaten är som arbetsgivare i olika delar av landet. Omvänt, hur icke-viktig arbetsgivare statsapparaten är i somliga kommuner, så är detta mått inget bra.

Resultatet publicerades i Dagens Samhälle nr 8 den 28 februari 2019. Artikeln finna [att läsa på webben här!](https://www.dagenssamhalle.se/nyhet/kommuner-som-lever-pa-offentlig-sektor-26466)

In [20]:
df = pd.read_csv('data/kom_reg.csv',sep=',',encoding='iso-8859-10')

In [21]:
df = pd.concat([pd.read_csv('data/samtliga.csv',encoding='iso-8859-10'),df]).fillna('totalt')

In [22]:
df.columns = ['antal','vars','kommun']

In [23]:
df.vars.unique()

array(['totalt', 'primärkommunal förvaltning', 'landsting',
       'kommunalt ägda företag och organisationer'], dtype=object)

Ovan ser vi de variabler som vi valt att kika närmare på. Det är alltså de som jobbar inom "primärkommunal förvaltning", "kommunlt ägda företag och organisationer" samt "landsting" som vi väljer att se som lokal offentliganställning. Statliga tjänster har vi valt bort. Det här har också kollats med SCB, Sofia Löfgren (010-479 65 86) som står som kontaktperson till statistiken på RAMS. Hon framhöll att jag hade valt att fokusera på rätt saker! Här finns rapporten som förklarar sektorsindelningen: https://www.scb.se/contentassets/99af4dcf7296448db1386574e1aa6b9b/mis2014-1.pdf

In [24]:
df.loc[df['vars']!='totalt','vars'] = 'offentliga'

In [25]:
df = df.groupby(['kommun','vars']).sum()\
       .reset_index()\
       .pivot_table(index='kommun',
                    columns='vars',
                    values='antal')\
       .reset_index()

In [26]:
df['andel_offentliganställda'] = ((df['offentliga']/df['totalt'])*100).round(1)

In [27]:
df['kommun'] = df['kommun'].apply(lambda x: x[5:].strip())

In [28]:
if not os.path.isdir('res'):
    os.makedirs('res')

In [29]:
df.sort_values('andel_offentliganställda',
               ascending=False)\
  .reset_index(drop=True)\
  .to_excel('res/andel_offtl_anställda_2017.xlsx',
            index=False)

In [30]:
df.sort_values('andel_offentliganställda',ascending=False).tail()

vars,kommun,offentliga,totalt,andel_offentliganställda
18,Nacka,6808,51040,13.3
20,Solna,5815,44035,13.2
21,Lidingö,2870,22315,12.9
13,Täby,3953,34759,11.4
14,Danderyd,1563,15050,10.4


Ovan ser vi exempel på att Stockholmskommuner alla kommer i botten av lista. Vi valde att inte publicera dessa.

### Flera år

Här kommer en funktion som gör det samma som ovan, fast vi kan själv välja vilket år vi vill kolla på (finns 2008-2017). Används nedan för att se ifall förändringen över tid är anmärkningsvärd. Men tycker nog inte den va det..

In [16]:
def formatter(year):
    
    df = pd.read_csv('data/of_allaår.csv',sep=',',encoding='iso-8859-10')
    df = df.loc[df['år']==year]
    tmp = pd.read_csv('data/totalt_allaår.csv',encoding='iso-8859-10')
    tmp = tmp.loc[tmp['år']==year]
    
    df = pd.concat([tmp,df]).fillna('totalt').drop('år',axis=1)
    
    df.columns = ['antal','vars','kommun']
    
    df.loc[df['vars']!='totalt','vars'] = 'offentliga'
    
    df = df.groupby(['kommun','vars']).sum()\
            .reset_index()\
            .pivot_table(index='kommun',
                         columns='vars',
                         values='antal').reset_index()
    
    df[f'andel_offentliganställda_{year}'] = ((df['offentliga']/df['totalt'])*100).round(1)
    
    df['kommun'] = df['kommun'].apply(lambda x: x[5:].strip())
    
    return df[['kommun',f'andel_offentliganställda_{year}']]

In [34]:
formatter(2017).head()

vars,kommun,andel_offentliganställda_2017
0,Upplands Väsby,14.5
1,Vallentuna,15.8
2,Österåker,15.5
3,Värmdö,16.7
4,Järfälla,18.8


In [17]:
df = formatter(2017).merge(formatter(2008),on='kommun',how='left')

In [18]:
df['utv'] = df['andel_offentliganställda_2017']-df['andel_offentliganställda_2008']

In [19]:
df.sort_values('andel_offentliganställda_2017',ascending=False).head()

vars,kommun,andel_offentliganställda_2017,andel_offentliganställda_2008,utv
274,Lycksele,41.9,43.2,-1.3
279,Överkalix,38.8,39.7,-0.9
67,Eksjö,38.4,36.5,1.9
272,Åsele,37.9,40.3,-2.4
271,Vilhelmina,37.4,37.9,-0.5


Vi tyckte att utvecklingen inte var särskilt anmärkningsvärd och struntade därför i detta i publiceringen. Vinkeln blev rak: var är det störst andel idag?